<a href="https://colab.research.google.com/github/jjossie/jbeds-road-signs/blob/main/Erik_GANs_Inception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Note: After you run this cell, the training and test data will be available in
# the file browser. (Click the folder icon on the left to view it)
#
# If you don't see the data after the cell completes, click the refresh button
# in the file browser (folder icon with circular arrow)

# First, let's download and unzip the data
!echo "Downloading files..."
!wget -q https://github.com/byui-cse/cse450-course/raw/master/data/roadsigns/training1.zip
!wget -q https://github.com/byui-cse/cse450-course/raw/master/data/roadsigns/training2.zip
!wget -q https://github.com/byui-cse/cse450-course/raw/master/data/roadsigns/test.zip
!wget -q https://github.com/byui-cse/cse450-course/raw/master/data/roadsigns/test_partial.zip
!wget -q https://github.com/byui-cse/cse450-course/raw/master/data/roadsigns/test_classes_partial.csv

!echo "Unzipping files..."
!unzip -q /content/training1.zip
!unzip -q /content/training2.zip
!unzip -q /content/test.zip
!unzip -q /content/test_partial.zip

# Combine the two traning directories
!echo "Merging training data..."
!mkdir /content/training
!mv /content/training1/* /content/training
!mv /content/training2/* /content/training

# Cleanup
!echo "Cleaning up..."
!rmdir /content/training1
!rmdir /content/training2
!rm training1.zip
!rm training2.zip
!rm test.zip
!rm test_partial.zip

!echo "Data ready."

Unzipping files...
Merging training data...
Cleaning up...
Data ready.


In [ ]:
# Import libraries
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
from keras import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.metrics import Precision, Recall
from sklearn.model_selection import train_test_split
from keras.applications import InceptionV3
from keras.applications import VGG16
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

In [ ]:

# Define a function to add random noise to an image
def add_random_noise(image):
    noise = np.random.normal(loc=0, scale=0.1, size=image.shape)  # Gaussian noise with mean 0 and standard deviation 0.1
    noisy_image = image + noise
    return np.clip(noisy_image, 0, 1)  # Clip values to ensure they are within the valid range [0, 1]


In [ ]:
# We're using keras' ImageDataGenerator class to load our image data.
# See (https://keras.io/api/preprocessing/image/#imagedatagenerator-class) for details
#
# A couple of things to note:
# 1. We're specifying a number for the seed, so we'll always get the same shuffle and split of our images.
# 2. Class names are inferred automatically from the image subdirectory names.
# 3. We're splitting the training data into 80% training, 20% validation.


training_dir = '/content/training/'
image_size = (224, 224)

# Split up the training data images into training and validations sets
# We'll use and ImageDataGenerator to do the splits
# ImageDataGenerator can also be used to do preprocessing and agumentation on the files as can be seen with rescale

train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,  # Random rotation within ±20 degrees
        zoom_range=0.2,     # Random zooming within ±20%
        width_shift_range=0.2,  # Random horizontal shifting
        height_shift_range=0.2, # Random vertical shifting
        preprocessing_function=add_random_noise,  # Add Gaussian noise
        validation_split=.2
        )
validation_datagen = ImageDataGenerator(
        rescale=1./255,
        # rotation_range=20,  # Random rotation within ±20 degrees
        # zoom_range=0.2,     # Random zooming within ±20%
        # width_shift_range=0.2,  # Random horizontal shifting
        # height_shift_range=0.2, # Random vertical shifting
        # preprocessing_function=add_random_noise,  # Add Gaussian noise
        validation_split=.2
        )

train_generator = train_datagen.flow_from_directory(
        training_dir,
        target_size = image_size,
        subset="training",
        batch_size=32,
        class_mode='categorical',
        seed=42,shuffle=True)
validation_generator = validation_datagen.flow_from_directory(
        training_dir,
        target_size=image_size,
        batch_size=32,
        class_mode='categorical',
        subset="validation",
        seed=42)

Found 31368 images belonging to 43 classes.
Found 7841 images belonging to 43 classes.


In [ ]:
# # GAN

# # Create the generator model
# generator_model = tf.keras.Sequential([
#     tf.keras.layers.Dense(256 * 7 * 7, activation='relu', input_dim=100),
#     tf.keras.layers.Reshape((7, 7, 256)),
#     tf.keras.layers.Conv2DTranspose(128, kernel_size=5, strides=2, padding='same', activation='relu'),
#     tf.keras.layers.Conv2DTranspose(64, kernel_size=5, strides=2, padding='same', activation='relu'),
#     tf.keras.layers.Conv2DTranspose(32, kernel_size=5, strides=2, padding='same', activation='relu'),
#     tf.keras.layers.Conv2DTranspose(3, kernel_size=5, strides=2, padding='same', activation='sigmoid')
# ])

# # Create the discriminator model
# discriminator_model = tf.keras.Sequential([
#     tf.keras.layers.Conv2D(32, kernel_size=5, strides=2, padding='same', activation='relu', input_shape=(7, 7, 256)),
#     tf.keras.layers.Conv2D(64, kernel_size=5, strides=2, padding='same', activation='relu'),
#     tf.keras.layers.Conv2D(128, kernel_size=5, strides=2, padding='same', activation='relu'),
#     tf.keras.layers.Conv2D(256, kernel_size=5, strides=2, padding='same', activation='relu'),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(1, activation='sigmoid')
# ])

# # Create the combined GAN model
# combined_model = tf.keras.Sequential([
#     generator_model,
#     discriminator_model
# ])

# # Compile the discriminator model
# discriminator_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# # Compile the combined GAN model
# combined_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# # Train the discriminator and generator iteratively
# for epoch in range(5):
#     # Train the discriminator
#     discriminator_model.fit(train_generator, epochs=1)

#     # Train the generator (while keeping the discriminator weights fixed)
#     combined_model.layers[0].trainable = True  # Enable training for the generator
#     combined_model.layers[1].trainable = False  # Disable training for the discriminator
#     combined_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
#     combined_model.fit(train_generator, epochs=1)

#     # Disable training for the generator
#     combined_model.layers[0].trainable = False
#     combined_model.layers[1].trainable = True


In [ ]:
# Load the pre-trained VGG16 model (excluding the top layer)
inc_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# # Freeze the base model layers
# for layer in inc_model.layers:
#     layer.trainable = False

# Create your CNN model based on VGG16
model = Sequential()
model.add(vgg_model)
model.add(Flatten())
model.add(Dense(43, activation='softmax'))  # 43 is the number of classes in this example

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model using the generator
training_history = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // train_generator.batch_size,
        epochs=6,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // validation_generator.batch_size
        )

Epoch 1/6
821/980 [========================>.....] - ETA: 1:41 - loss: 3.5227 - accuracy: 0.0543

In [ ]:
test_dir = '/content/'

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        test_dir,
        classes=['test_partial'],
        target_size=image_size,
        class_mode='categorical',
        shuffle=False)
probabilities = model.predict(test_generator)

# Load the ground truth labels from the CSV file
test_labels = pd.read_csv('test_classes_partial.csv')

# Get the predicted labels from the model
predicted_labels = [np.argmax(probas) for probas in probabilities]

# Get the true labels from the ground truth
true_labels = test_labels['ClassId'].to_list()

Found 201 images belonging to 1 classes.
7/7 [==============================] - 3s 401ms/step


In [ ]:
# Compute the metrics
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average='weighted', zero_division=0)
recall = recall_score(true_labels, predicted_labels, average='weighted', zero_division=0)
f1 = f1_score(true_labels, predicted_labels, average='weighted')

# Print the metrics
print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)

Accuracy: 0.03980099502487562
Precision: 0.001584119204970174
Recall: 0.03980099502487562
F1 Score: 0.0030469661263062674
